In [58]:
import json
import numpy as np

total_data = []
with open('data/shbank_company_split.txt', 'r', encoding='utf-8') as f:
    for line in f:
        l = json.loads(line)
        total_data.append(l)
# sample_data = np.random.choice(total_data, int(len(total_data) * 0.01), replace=False)

In [59]:
non_place = []
non_brand = []
non_trade = []
non_suffix = []
for idx, item in zip(range(len(total_data)), total_data):
    if 'place' not in item['label'] or item['label']['place'] == []:
        non_place.append(idx)
    if 'brand' not in item['label'] or item['label']['brand'] == []:
        non_brand.append(idx)
    if 'trade' not in item['label'] or item['label']['trade'] == []:
        non_trade.append(idx)
    if 'suffix' not in item or item['label']['suffix'] == []:
        non_suffix.append(idx)

In [60]:
norma_idx = set(range(len(total_data))) - (set(non_place) | set(non_brand) | set(non_trade))  #  字段全的数据

In [61]:
print(len(non_place))
print(len(non_brand))
print(len(non_trade))
print(len(norma_idx))

7935
1861
4432
89286


In [43]:
# idx_list = list(set(non_place) & set(non_brand) & set(non_trade))
# idx_list.sort()
# idx_list.reverse()

In [48]:
# idx_list = list(set(non_place) & set(non_brand))
# idx_list.sort()
# idx_list.reverse()

In [49]:
# for idx in idx_list:
#     total_data.pop(idx)

In [ ]:
# for idx in non_suffix:
#     total_data.pop(idx)

In [67]:
norma_idx_choice = np.random.choice(list(norma_idx), int(len(total_data) * 0.05), replace=False)
non_place_choice = np.random.choice(list(non_place), int(len(non_place) * 0.1), replace=False)
non_brand_choice = np.random.choice(list(non_brand), int(len(non_brand) * 0.1), replace=False)
non_trade_choice = np.random.choice(list(non_trade), int(len(non_trade) * 0.1), replace=False)
non_place_trade = list(set(non_place) & set(non_trade))
non_place_trade_choice = np.random.choice(non_place_trade, int(len(non_place_trade) * 0.1), replace=False)
non_brand_trade = list(set(non_brand) & set(non_trade))
non_brand_trade_choice = np.random.choice(non_brand_trade, int(len(non_brand_trade) * 0.1), replace=False)

In [68]:
samples_id = list(set(norma_idx_choice) | set(non_place_choice) | set(non_brand_choice) | set(non_trade_choice) \
                | set(non_place_trade_choice) | set(non_brand_trade_choice))

In [69]:
import random
random.shuffle(samples_id)

In [70]:
from tqdm import tqdm
with open('data/shbank_company_name_sample.txt', 'w', encoding='utf-8') as f:
    for idx in tqdm(samples_id):
        # if total_data[idx]['text'] in check_name:
        #     continue 
        f.write(json.dumps(total_data[idx], ensure_ascii=False) + '\n')

100%|██████████| 6669/6669 [00:00<00:00, 76740.44it/s]


In [72]:
import json
check_name = set()
with open('data/name-sample-check.json', 'r', encoding='utf-8') as f:
    for line in f:
        try:
            l = json.loads(line.strip())
            check_name.add(l['text'])
        except:
            print(line)

In [73]:
len(check_name)

1498

In [75]:
samples = []
with open('data/name-sample-check.json', 'r', encoding='utf-8') as f:
    for line in f:
        l = json.loads(line.strip())
        samples.append(l)

for item in samples:
    for k in ['place', 'brand', 'trade', 'suffix']:
        if k not in item['label'].keys():
            continue
        for l in item['label'][k]:
            if item['text'][l[1]: l[2]] != l[0]:
                print(item['text'], item['text'][l[1]: l[2]], l)

In [76]:
f = open('bert-base-chinese/vocab.txt', 'r', encoding='utf-8')
words = f.readlines()
words_c = set()
for w in words:
    words_c.add(w.strip())
    
# for item in samples:
#     if len(set(item['text']) - words_c) > 0:
#         print(item['text'], set(item['text']) - words_c)
#     print(tokenizer.tokenize(item['text']))

In [77]:
from utils import util
import torch
import config
import logging
import numpy as np
# from data_process import Processor
from utils.data_loader import NERDataset
from train_file.model import BertNER
from train_file.train import train, evaluate
from transformers import (
  BertTokenizerFast,
)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers.optimization import get_cosine_schedule_with_warmup, AdamW

import warnings
import os
warnings.filterwarnings('ignore')
import pickle
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [78]:
filename = config.model_dir
batch_size=config.batch_size
device = config.device
if not os.path.exists(filename):               #判断文件夹是否存在
    os.makedirs(filename)                       #新建文件夹
"""train the model"""
# set the logger
util.set_logger(config.log_dir)
logging.info("device: {}".format(device))
logging.info("batch_size: {}".format(batch_size))
   

device: cuda:0
batch_size: 64


In [79]:
import json
class Processor:
    def __init__(self, config):
        self.data_dir = config.data_dir
        self.config = config

    def process(self):
        """
        process train and test data
        """
        for file_name in self.config.files:
#             try:
            self.preprocess(file_name)
#             except:
#                 continue

    def preprocess(self, mode):
        """
        params:
            words：将json文件每一行中的文本分离出来，存储为words列表
            labels：标记文本对应的标签，存储为labels
        examples:
            words示例：['生', '生', '不', '息', 'C', 'S', 'O', 'L']
            labels示例：['O', 'O', 'O', 'O', 'B-game', 'I-game', 'I-game', 'I-game']
        """
        bad_tokens = [' ','\n',' ',' ','​','  ',' �','   ',' 　 ',' � ',' ﻿ ',' 	 ',' 	 ',' � ','   ',' ﻿ ',' 　 ',' 　 ',
                      ' 　 ',' 　 ',' 　 ',' ‍ ',' 　 ','   ','   ',' 　 ',' ﻿ ','   ']
        input_dir = self.data_dir + str(mode) + '.json'
        output_dir = self.data_dir + str(mode) + '.npz'
#         if os.path.exists(output_dir) is True:
#             return
        word_list = []
        label_list = []
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
        with open(input_dir, 'r', encoding='utf-8') as f:
            for line in tqdm(f):
                line = line.replace('\\xa0','')
                # loads()：用于处理内存中的json对象，strip去除可能存在的空格
                json_line = json.loads(line.strip())
                text = json_line['text']
                text = text.lower().replace("“",'"').replace("”",'"')
                words = tokenizer.tokenize(text)
                   
                # 如果没有label，则返回None
                label_entities = json_line.get('label', None)
                labels = ['O'] * len(words)
                flag = 0
                for key, value in label_entities.items():
                    if key == 'symbol':
                        continue
                    for item in value:
                        sub_name = item[0]
                        start_index = item[1]
                        end_index = item[2]
                        # if ''.join(words[item[1]:item[2]]) == sub_name:
        #               print(sub_name, ''.join(words[start_index:end_index + 1]))
                        flag = 1
                        # if len(sub_name) == 1:
                        #     labels[start_index] = 'S-' + key
                        # else:
                        labels[start_index] = 'B-' + key
                        labels[start_index + 1:end_index] = ['I-' + key] * (len(sub_name) - 1)
                if len(words) < 510 and flag == 1:
        #             print(words,labels)
                    word_list.append(words)
                    label_list.append(labels)

            # 保存成二进制文件
#         print(word_list,label_list)
        np.savez(output_dir, words=np.asanyarray(word_list, dtype=object), \
                 labels=np.asanyarray(label_list, dtype=object), dtype=object)
        logging.info("--------{} data process DONE!--------".format(mode))


In [80]:
def dev_split(dataset_dir):
    """split dev set"""
    data = np.load(dataset_dir, allow_pickle=True)
    words = data["words"]
    labels = data["labels"]
    x_train, x_dev, y_train, y_dev = train_test_split(words, labels, test_size=config.dev_split_size, random_state=0)
    return x_train, x_dev, y_train, y_dev


def load_dev(mode):
    if mode == 'train':
        # 分离出验证集
        word_train, word_dev, label_train, label_dev = dev_split(config.train_dir)
    elif mode == 'test':
        train_data = np.load(config.train_dir, allow_pickle=True)
        dev_data = np.load(config.test_dir, allow_pickle=True)
        word_train = train_data["words"]
        label_train = train_data["labels"]
        word_dev = dev_data["words"]
        label_dev = dev_data["labels"]
    elif mode == 'predict':
        train_data = np.load(config.train_dir, allow_pickle=True)
        predict_data = np.load(config.predict_dir, allow_pickle=True)
        word_train = train_data["words"]
        label_train = train_data["labels"]
        word_dev = predict_data["words"]
        label_dev = predict_data["labels"]
    else:
        word_train = None
        label_train = None
        word_dev = None
        label_dev = None
    return word_train, word_dev, label_train, label_dev

In [81]:
# %%time
processor = Processor(config)
processor.process()
logging.info("--------Process Done!--------")


1501it [00:00, 10426.47it/s]
--------name-sample-check data process DONE!--------
--------Process Done!--------


In [82]:
# 分离出验证集
word_train, word_dev, label_train, label_dev = load_dev('train')
logging.info("--------load_dev !--------")
# build dataset
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
train_dataset = NERDataset(word_train, label_train, config, tokenizer)
dev_dataset = NERDataset(word_dev, label_dev, config, tokenizer)
with open(filename+'train_dataset.pkl', 'wb') as f:
    pickle.dump(train_dataset, f)
with open(filename+'dev_dataset.pkl', 'wb') as f:
    pickle.dump(dev_dataset, f)
logging.info("--------Dataset Build!--------")
            

--------load_dev !--------
100%|██████████| 151/151 [00:00<00:00, 55173.79it/s]
--------Dataset Build!--------


In [83]:
for i in range(len(label_train)):
    if len(word_train[i]) != len(label_train[i]):
        print(word_train[i], label_train[i])

In [84]:
# get dataset size
train_size = len(train_dataset)
print(train_size)
# build data_loader
train_loader = DataLoader(train_dataset, batch_size=config.batch_size,
                            shuffle=True, collate_fn=train_dataset.collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size,
                        shuffle=True, collate_fn=dev_dataset.collate_fn)
with open(filename+'train_loader'+str(batch_size)+'.pkl', 'wb') as f:
        pickle.dump(train_loader, f)
with open(filename+'dev_loader'+str(batch_size)+'.pkl', 'wb') as f:
    pickle.dump(dev_loader, f)
logging.info("--------Get Dataloader!--------")

--------Get Dataloader!--------


1350


In [85]:
check_model_dir = ''
# Prepare model
if check_model_dir != '':
    model = BertNER.from_pretrained(check_model_dir)
    logging.info("--------Load model from {}--------".format(check_model_dir))
else:
    model = BertNER.from_pretrained('bert-base-chinese',num_labels=len(config.label2id))
    logging.info("--------Create model from {}--------".format('bert-base-chinese'))
model.to(device)
# train_loader.to(device)
# dev_loader.to(device)
# Prepare optimizer


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertNER: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertNER were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['bilstm.weight_ih_l1', 'bilstm.bias_ih_l1

BertNER(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [86]:
if config.full_fine_tuning:
    # model.named_parameters(): [bert, bilstm, classifier, crf]
    bert_optimizer = list(model.bert.named_parameters())
    # lstm_optimizer = list(model.bilstm.named_parameters())
    classifier_optimizer = list(model.classifier.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in bert_optimizer if not any(nd in n for nd in no_decay)],
            'weight_decay': config.weight_decay},
        {'params': [p for n, p in bert_optimizer if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0},
        # {'params': [p for n, p in lstm_optimizer if not any(nd in n for nd in no_decay)],
            # 'lr': config.learning_rate * 20, 'weight_decay': config.weight_decay},
        # {'params': [p for n, p in lstm_optimizer if any(nd in n for nd in no_decay)],
            # 'lr': config.learning_rate * 20, 'weight_decay': 0.0},
        {'params': [p for n, p in classifier_optimizer if not any(nd in n for nd in no_decay)],
            'lr': config.learning_rate * 20, 'weight_decay': config.weight_decay},
        {'params': [p for n, p in classifier_optimizer if any(nd in n for nd in no_decay)],
            'lr': config.learning_rate * 20, 'weight_decay': 0.0},
        {'params': model.crf.parameters(), 'lr': config.learning_rate * 20}
    ]
# only fine-tune the head classifier
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=config.learning_rate, correct_bias=False)

train_steps_per_epoch = train_size // config.batch_size
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=(config.epoch_num // 10) * train_steps_per_epoch,
                                            num_training_steps=config.epoch_num * train_steps_per_epoch)
# Train the model
logging.info("--------Start Training!--------")
train(train_loader, dev_loader, model, optimizer, scheduler, config.model_dir)


--------Start Training!--------
100%|██████████| 22/22 [00:02<00:00, 10.13it/s]
Epoch: 1, train loss: 1545.6822683160956
100%|██████████| 3/3 [00:00<00:00, 16.26it/s]
Epoch: 1, dev loss: 1193.7692464192708, f1 score: 0.08695652173913043
--------Save best model!--------
--------Save best loss model!--------
100%|██████████| 22/22 [00:02<00:00,  9.89it/s]
Epoch: 2, train loss: 1350.6452945362437
100%|██████████| 3/3 [00:00<00:00, 16.66it/s]
Epoch: 2, dev loss: 950.6960042317709, f1 score: 0.304648862512364
--------Save best model!--------
--------Save best loss model!--------
100%|██████████| 22/22 [00:02<00:00, 10.12it/s]
Epoch: 3, train loss: 1050.9796537919478
100%|██████████| 3/3 [00:00<00:00, 16.93it/s]
Epoch: 3, dev loss: 714.463124593099, f1 score: 0.6486956521739131
--------Save best model!--------
--------Save best loss model!--------
100%|██████████| 22/22 [00:02<00:00, 10.13it/s]
Epoch: 4, train loss: 798.6135052767667
100%|██████████| 3/3 [00:00<00:00, 17.01it/s]
Epoch: 4, de

In [87]:
torch.save(model, config.model_dir + 'last_model/' + 'model.bin')

In [88]:
from utils.metrics import f1_score, bad_case, predict_output, predict_output_pre
# def evaluate_pre(dev_loader, model, tokenizer,mode='dev',flag=1):
# set model to evaluation mode
mode = 'predict'
model.eval()
id2label = config.id2label
true_tags = []
pred_tags = []
sent_data = []
dev_losses = 0
len_dev_loader = 0
with torch.no_grad():
    for idx, batch_samples in enumerate(tqdm(dev_loader)):
        batch_data, batch_token_starts, batch_tags = batch_samples
        if mode == 'test' or mode == 'predict':
            sent_data.extend([[tokenizer.convert_ids_to_tokens(idx.item()) for idx in indices
                               if (idx.item() > 0 and idx.item() != 101)] for indices in batch_data])
        batch_masks = batch_data.gt(0)  # get padding mask, gt(x): get index greater than x
        label_masks = batch_tags.gt(-1)  # get padding mask, gt(x): get index greater than x
        len_dev_loader += 1
        # (batch_size, max_len, num_labels)
        batch_output = model((batch_data, batch_token_starts),
                             token_type_ids=None, attention_mask=batch_masks)[0]
        # (batch_size, max_len - padding_label_len)
        batch_output = model.crf.viterbi_decode(batch_output, mask=label_masks)
        # (batch_size, max_len)
        batch_tags = batch_tags.to('cpu').numpy()
        pred_tags.extend([[id2label.get(idx) for idx in indices] for indices in batch_output])
        # (batch_size, max_len - padding_label_len)
        true_tags.extend([[id2label.get(idx) for idx in indices if idx > -1] for indices in batch_tags])

# print(len(pred_tags), len(true_tags))
# print(pred_tags, true_tags)
assert len(pred_tags) == len(true_tags)
if mode == 'test' or mode == 'predict':
    assert len(sent_data) == len(true_tags)

# logging loss, f1 and report
metrics = {}

if mode == 'predict' or mode == 'test':
    # print(pred_tags,sent_data)
    all_end=predict_output_pre(pred_tags, sent_data)
    
metrics['loss'] = float(dev_losses) / len_dev_loader
#     if flag==0:
#         return metrics,all_end
#     else:
#         return metrics


100%|██████████| 3/3 [00:00<00:00, 12.84it/s]
--------Predict finished !--------


In [89]:
all_end

[['灌南苏盛能源有限公司',
  {'place': [['灌南', 0, 2]],
   'brand': [['苏盛', 2, 4]],
   'trade': [['能源', 4, 6]],
   'suffix': [['有限公司', 6, 10]]}],
 ['康达医疗康复设备有限公司',
  {'brand': [['康达', 0, 2]],
   'trade': [['医疗', 2, 4], ['康复设备', 4, 8]],
   'suffix': [['有限公司', 8, 12]]}],
 ['固原市第四建筑工程公司',
  {'place': [['固原市', 0, 3]],
   'brand': [['第四', 3, 5]],
   'trade': [['建筑工程', 5, 9]],
   'suffix': [['公司', 9, 11]]}],
 ['长沙新兴服务劳务公司',
  {'place': [['长沙', 0, 2]],
   'brand': [['新兴', 2, 4]],
   'trade': [['服务劳务', 4, 8]],
   'suffix': [['公司', 8, 10]]}],
 ['四川新能源开发公司',
  {'place': [['四川', 0, 2]],
   'trade': [['新能源开发', 2, 7]],
   'suffix': [['公司', 7, 9]]}],
 ['昆山正亚电子有限公司',
  {'place': [['昆山', 0, 2]],
   'brand': [['正亚', 2, 4]],
   'trade': [['电子', 4, 6]],
   'suffix': [['有限公司', 6, 10]]}],
 ['成都天立化工科学有限公司',
  {'place': [['成都', 0, 2]],
   'brand': [['天立', 2, 4]],
   'trade': [['化工科学', 4, 8]],
   'suffix': [['有限公司', 8, 12]]}],
 ['南京平安总公司',
  {'place': [['南京', 0, 2]],
   'brand': [['平安', 2, 4]],
   'suffix': [['总公司', 4, 7

In [90]:
from utils.metrics import convert_to_dict_pre

class CompanyPredict():
    def __init__(self, model, config, tokenizer, word_pad_idx = 0) -> None:
        self.model = model
        self.tokenizer = tokenizer
        # BertTokenizer.from_pretrained(config.bert_model, do_lower_case=True)
        self.label2id = config.label2id
        self.id2label = {_id: _label for _label, _id in list(config.label2id.items())}
        self.word_pad_idx = word_pad_idx
        self.device = config.device

    def data_process(self, company_names):
        sentences_ori = []
        sentences = []
        for company in company_names:
            words = ['[CLS]'] + tokenizer.tokenize(company)
            sentences_ori.append(words)
            word_lens = []
            for w in words[1:]: #防止有英文
                word_lens.append(1) #len(w))
            token_start_idxs = 1 + np.cumsum([0] + word_lens[:-1])
            sentences.append((tokenizer.convert_tokens_to_ids(words), token_start_idxs))
        # print(list(zip(sentences_ori[1][1:], sentences[1][-1])))

        # print(list(zip(words[1:], sentences[-1][-1])))
        # batch length
        batch_len = len(sentences)

        # compute length of longest sentence in batch
        max_len = max([len(s[0]) for s in sentences])

        # padding data 初始化
        batch_data = self.word_pad_idx * np.ones((batch_len, max_len))
        max_label_len = 0
        batch_label_starts = []
        sent_data = []
        # padding and aligning
        for j in range(batch_len):
            cur_len = len(sentences[j][0])
            batch_data[j][:cur_len] = sentences[j][0] #sentence
            # 找到有标签的数据的index（[CLS]不算）
            label_start_idx = sentences[j][-1] # word start index
            label_starts = np.zeros(max_len)
            label_starts[[idx for idx in label_start_idx if idx < max_len]] = 1
            batch_label_starts.append(label_starts)
            max_label_len = max(int(sum(label_starts)), max_label_len)

        # convert data to torch LongTensors
        batch_data = torch.tensor(np.array(batch_data), dtype=torch.long)
        batch_label_starts = torch.tensor(np.array(batch_label_starts), dtype=torch.long)
        #         print(batch_data.shape, batch_labels.shape)
        #         print(batch_data, batch_labels)
        # shift tensors to GPU if available
        batch_data, batch_label_starts = batch_data.to(self.device), batch_label_starts.to(self.device)
        
        return batch_data, batch_label_starts, sentences_ori
    
    def predict(self, batch_data, batch_label_starts, sentences):
        batch_masks = batch_data.gt(0)

        batch_output = self.model((batch_data, batch_label_starts),
                token_type_ids=None, attention_mask=batch_masks)

        embedding = batch_output[1]
        batch_output = batch_output[0]

        batch_output = self.model.crf.viterbi_decode(batch_output, mask=batch_masks[:, 1:])

        pred_tags = [[self.id2label.get(idx) for idx in indices] for indices in batch_output]
        
        all_end = []
        for pred_label, senten in zip(pred_tags, sentences):
            res = convert_to_dict_pre(senten[1:], pred_label)
            for label in ['place', 'brand', 'trade', 'suffix']:
                if label not in res.keys():
                    res[label] = []
                
            all_end.append(res)
        # all_end = predict_output_pre(pred_tags, sentences_ori)
        return all_end, pred_tags, embedding

In [91]:
import torch
import json
from transformers import (
  BertTokenizerFast,
)
import config
from tqdm import tqdm 
import numpy as np
from zhconv import convert 
tokenizer = BertTokenizerFast.from_pretrained('./bert-base-chinese')
model = torch.load('experiments/s_model/best_model/model.bin')
# model = torch.load('experiments/s_model/last_model/model.bin')

In [93]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained('glm-large-chinese', trust_remote_code=True)
# model = AutoModelForSeq2SeqLM.from_pretrained('glm-large-chinese', trust_remote_code=True)
# model = model.half().cuda()
# model.eval()
# # Inference
# inputs = tokenizer("Ng is an adjunct professor at [MASK] (formerly associate professor and Director of its Stanford AI Lab or SAIL ). Also a pioneer in online education, Ng co-founded Coursera and deeplearning.ai.", return_tensors="pt")
# inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=512)
# inputs = inputs.to('cuda')
# outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
# print(tokenizer.decode(outputs[0].tolist()))
# # Training
# inputs = tokenizer(    ["Tsinghua University is located in [MASK].", "One minus one equals zero, is it correct? Answer: [MASK]"],    return_tensors="pt", padding=True)
# inputs = tokenizer.build_inputs_for_generation(inputs, targets=["Beijing", "No"], max_gen_length=8, padding=False)
# inputs = inputs.to('cuda')
# outputs = model(**inputs)
# loss = outputs.losslogits = outputs.logits

In [94]:
file_dirs = os.listdir('./data/')
del_dir = []
for file_dir in file_dirs:
    if not file_dir.isnumeric():
        del_dir.append(file_dir)
file_dirs = list(set(file_dirs) - set(del_dir))


In [95]:
file_dirs.sort(reverse=True)


In [101]:
def is_chinese(string):
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

In [102]:
import re
company_ids = []
company_names = []
with open('blacklist_data/20231105/HMD_SA29_RECORD_LIST_20231105.dat', 'r', encoding='utf-8') as f:
    for line in f:
        l = line.strip().split('@!@')
        l_id = l[0]
        l_name = convert(l[1].strip().replace("“",'"').replace("”",'"'), 'zh-cn')
        if not is_chinese(l_name):
            continue
        l_name = re.sub(r'[^\u4e00-\u9fffa-zA-Z0-9()（）"\'“”‘’]', ' ', l_name.lower())
        company_ids.append(l_id)
        company_names.append(l_name)


In [103]:
# company_names = []
# with open('data/name-sample-check.json', 'r', encoding='utf-8') as f:
#     for line in f:
#         l = json.loads(line.strip())
#         company_names.append(l['text'].strip().replace("“",'"').replace("”",'"'))
# company_names = ['福建省福州高速公路管理公司', 
#                  '亚马逊信息服务(北京)有限公司',
#                  '汕头嘉祥塑料制品厂有限公司', 
#                  'happy昱景科技股份有限公司', 
#                  '雷蒙药品经销有限责任公司',
#                  '上海农商银行',
#                  '中国农业银行',
#                  '温州银行"金鹿理财－普惠颐养"２１０５０号开放式净值型银行理财计划',
#                  '卓越成长定增基金1号',
#                  '上海雄艮企业管理中心（有限合伙）']
# company_names = ['上行', '工商银行', '农行', '上海银行', '上海农商银行', '中国农业银行', '浦发银行', '阿里巴巴', '阿里', '腾讯', '邦盛科技', '银行']

In [104]:
int(len(company_names) / 128) + 1

43

In [105]:
# model_dir = config.model_dir + '/best'
# model = torch.load('experiments/s_model/model.bin')
# # BertNER.from_pretrained(model_dir)
# model.to(config.device)
CPredict = CompanyPredict(model, config, tokenizer)

results = []
part = 128
iter_num = int(len(company_names) / part) + 1
embeddings = []
for i in tqdm(range(iter_num)):
    start = i * part 
    if i == iter_num -1:
        end = len(company_names)
    else:
        end = (i + 1) * part
    batch_data, batch_label_starts, sentences_ori = \
        CPredict.data_process(company_names[start:end])
    all_end, pred_tags, _ = CPredict.predict(batch_data, batch_label_starts, sentences_ori)
    results.extend(all_end)
    # embeddings.extend(embedding)

100%|██████████| 43/43 [00:04<00:00, 10.20it/s]


In [106]:
results_ori = []
for i in range(len(results)):
    results_ori.append({'id': company_ids[i], 'text': company_names[i], 'label': results[i]})

In [107]:
results = results_ori

In [108]:
results[10]

{'id': 'A143278',
 'text': '上行普惠十三公司',
 'label': {'brand': [['上行普惠', 0, 4], ['十三', 4, 6]],
  'suffix': [['公司', 6, 8]],
  'place': [],
  'trade': []}}

In [109]:
import numpy as np

def calculate_levenshtein_distance(text1, text2):
    m, n = len(text1), len(text2)
    dp = np.zeros((m + 1, n + 1))

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1

    return dp[m][n]

In [114]:
from fuzzywuzzy import fuzz
string1 = "汽车"
string2 = "汽车制造"
sim_score = fuzz.ratio(string1, string2)
print(sim_score)

67


In [ ]:
# 富源县营上镇大坪煤矿
string1 = "大众"
string2 = "大坪"
sim_score = fuzz.ratio(string1, string2)
print(sim_score)

In [110]:
CPredict = CompanyPredict(model, config, tokenizer)

target_text = '科技中心'
# company_names = ['哈尔滨黄金自营有限公司', 
#                   '测试公司1', 
#                   '马宗仓自有资金有限公司', 
#                   '广西中金金属科技有限公司', 
#                   '柳州市元哥再生资源回收有限公司', 
#                   '温州银行"金鹿理财－普惠颐养"２１０５０号开放式净值型银行理财计划']
batch_data, batch_label_starts, sentences_ori = \
        CPredict.data_process([target_text])
all_end, pred_tags, embedding = CPredict.predict(batch_data, batch_label_starts, sentences_ori)

target = {'text': target_text, 'label': all_end[0]}
target_embedding = embedding[0]

In [111]:
target

{'text': '科技中心',
 'label': {'trade': [['科技', 0, 2]],
  'suffix': [['中心', 2, 4]],
  'place': [],
  'brand': []}}

In [112]:
from collections import defaultdict
# target = {'text': '同里',
#   'label': {'brand': [['同里', 0, 2]], 'place': [], 'trade': [], 'suffix': []}}
candidates = []
for result in results:
    score = defaultdict(float)

    # print(item['text'], target['text'],  len(set(item['text']) & set(target['text'])))
    if len(set(result['text']) & set(target['text'])) == 0:
        continue
    if len(set(result['text']) & set(target['text'])) == len(target['text']):
        score['score'] = 0
        candidates.append([result['id'], result['text'], score['score']])
        continue
    
    score['score'] = 0
    for label in ['place', 'trade', 'brand', 'suffix']:
        score[label] = 0
        if target['label'][label] == [] and result['label'][label] == []:
            continue

        if target['label'][label] == [] and result['label'][label] != []:
            # score[item['text']][label] += sum([len(candi[0]) for candi in item['label'][label]])
            # score[item['text']]['score'] += score[item['text']][label]
            continue

        if target['label'][label] != [] and result['label'][label] == []:
            score[label] += sum([len(candi[0]) for candi in target['label'][label]])
        else:
            target_text = ''.join([item[0] for item in target['label'][label]])
            candi = ''.join([item[0] for item in result['label'][label]])
            score[label] = calculate_levenshtein_distance(target_text, candi)
        
        if label == 'brand':
            score[label] = score[label] * 2
        if label == 'trade':
            score[label] = score[label] * 1.5

        score['score'] += score[label]

    # if score[item['text']]['score'] < len(target['text']):
    candidates.append([result['id'], result['text'], score['score']])

In [113]:
import pandas as pd
pd.DataFrame(candidates).sort_values(by=2)[:50]

,0,1,2
461,A142615-1146,上海克皮网络科技中心,0.0
829,A142613-0880,广州子纯科技商行,2.0
1094,A142615-0912,广州亦峰科技部,2.0
921,A142613-0284,广州地基科技商行,2.0
963,A142613-0204,广州裙利科技商行,2.0
663,A142613-0962,广州瑞正科技商行,2.0
951,A142613-0390,广州涛米科技店,2.0
98,dowjones836583,东土耳其斯坦信息中心,3.0
530,A142615-1152,白银区品格日化中心,3.0
801,A142615-1176,长沙酷峥物业中心,3.0


In [10]:
# import numpy as np
# def cal_cosine_similarity(vector1, vector2):
#     cos_sim = vector1.dot(vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
#     return cos_sim

In [23]:
text = '邯郸腾信广告有限公司'
batch_data, batch_label_starts, sentences_ori = \
        CPredict.data_process([text])
all_end, pred_tags, embedding = CPredict.predict(batch_data, batch_label_starts, sentences_ori)

target = all_end[0]
target_embedding = embedding[0]

In [24]:
target_embedding[2:4]

tensor([[-0.0977, -0.1592,  1.0096,  ..., -0.0745, -0.4246, -1.4060],
        [-0.3302, -0.4835,  0.1196,  ...,  1.0540,  0.6491, -0.2148]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [27]:
from collections import defaultdict

score = defaultdict(dict)
candidates = []
for i in range(len(results)):
    item = results[i]
    if len(set(item['text']) & set(target['text'])) == 0:
        continue
    score[item['text']]['score'] = 0
    for label in ['place', 'trade', 'brand', 'suffix']:
        score[item['text']][label] = 0
        if target['label'][label] == [] and item['label'][label] == []:
            continue

        if target['label'][label] == [] and item['label'][label] != []:
            # score[item['text']][label] += sum([len(candi[0]) for candi in item['label'][label]])
            # score[item['text']]['score'] += score[item['text']][label]
            continue

        if target['label'][label] != [] and item['label'][label] == []:
            # score[item['text']][label] += 0
            # score[item['text']]['score'] += score[item['text']][label]
            continue

        for target_text in target['label'][label]:
            for candi in item['label'][label]:
                score[item['text']][label] += cal_cosine_similarity(target_embedding[target_text[1]: target_text[2]].sum(0).cpu().detach().numpy(), \
                                                                embeddings[i][candi[1]: candi[2]].sum(0).cpu().detach().numpy())
        score[item['text']]['score'] += score[item['text']][label]

        candidates.append([item['text'], score[item['text']]['score']])

In [31]:
import pandas as pd
rr = pd.DataFrame(candidates)
rr.sort_values(by=1)

,0,1
274,邯郸腾信广告有限公司,0.0
698,陕西嘉禾广告有限公司,4.0
429,合肥十方广告有限公司,4.0
377,四川笑苑广告有限公司,4.0
824,上海森林广告有限公司,4.0
...,...,...
414,吉林市华阳机电设备有限公司,9.0
630,吉林省衡运文化传媒有限公司,9.0
633,台州市海森泰进出口有限公司,9.0
339,常州市精创塑料科技有限公司,9.0


In [36]:
target_embedding[target_text[1]: target_text[2]]

tensor([[ 0.0520, -0.0061, -0.0453,  ..., -0.0565,  0.0668,  0.0365],
        [-0.0328,  0.1007, -0.1391,  ..., -0.0254,  0.0855,  0.0592],
        [-0.0946,  0.1788, -0.1453,  ...,  0.0084,  0.0660,  0.0519],
        [-0.1386,  0.2256, -0.1158,  ...,  0.0175,  0.0366,  0.0286]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [112]:
a = embedding[-5].sum(0)
b = embedding[-4].sum(0)

In [113]:
cos_sim1 = cosine_similarity(a.cpu().detach().numpy().reshape(1, -1), b.cpu().detach().numpy().reshape(1, -1))

In [114]:
cos_sim1

array([[0.8863816]], dtype=float32)

In [33]:
a1 = embedding[0][:2].sum(0)
a2 = embedding[3].sum(0)
a3 = embedding[4].sum(0)

In [21]:
# with open('data/shbank_company_split.txt', 'w', encoding='utf-8') as f:
#     for res, com in zip(results, company_names):
#         res['text'] = com
#         f.write(json.dumps(res, ensure_ascii=False) + '\n')

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim1 = cosine_similarity(a1.cpu().detach().numpy().reshape(1, -1), a2.cpu().detach().numpy().reshape(1, -1))

In [35]:
cos_sim2 = cosine_similarity(a1.cpu().detach().numpy().reshape(1, -1), a3.cpu().detach().numpy().reshape(1, -1))
cos_sim3 = cosine_similarity(a2.cpu().detach().numpy().reshape(1, -1), a3.cpu().detach().numpy().reshape(1, -1))

In [1]:
from annoy import AnnoyIndex
import random
f = 768
t = AnnoyIndex(f, 'euclidean')

ModuleNotFoundError: No module named 'annoy'

In [37]:
cos_sim2

array([[0.28189948]], dtype=float32)

In [38]:
cos_sim3

array([[0.9458423]], dtype=float32)

In [31]:
import re
ss = re.sub(r'[^\u4e00-\u9fffa-zA-Z0-9()（）"\'“”‘’]', '', 'Hello世界(123)--（456）“”‘"\'#$%')

In [32]:
len(ss)

22

In [33]:
ss

'Hello世界(123)（456）“”‘"\''